In [7]:
import pandas as pd
import numpy as np
import json
from sklearn.cluster import SpectralClustering
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import os

platform = "twitter"
	# read narrative time series
df_event = pd.read_csv('./data_csv/'+platform+'_event.csv', header=0,index_col=0)

	# read GDELT time series
df_gdelt = pd.read_csv('./data_csv/gdelt.csv', header=0,dtype={'InfoID':str})
df_gdelt.set_index('InfoID', inplace=True)

	# read correlation
df_corr = pd.read_csv('./data_csv/corr_'+platform+'.csv', header=0,index_col=0)

infoIDs = sorted(df_event.index.values) # narrative
	# eventIDs = df_corr.columns.values # GDELT

df_gdelt = df_gdelt.sort_index()

	# Find the popular event ID we use as the input
active_event = df_gdelt[df_gdelt.sum(axis=1).gt(8000)].index.values

	# active_gdelt = list(active_gdelt)
df_gdelt = df_gdelt.loc[active_event,:]
df_corr = df_corr.loc[:,active_event]
df_corr[df_corr.lt(0.3)] = 0
print (df_corr.shape)
print (df_event.shape)


	# k+1,K+2,...,k+n
    
k=0
n=2
path = './train/'+platform+'/'+str(n)
if not os.path.exists(path):
	os.mkdir(path)

fx_train = open(os.path.join(path, 'x.csv'), 'w')
fi_train = open(os.path.join(path, 'ind.csv'), 'w')
fy_train = open(os.path.join(path, 'y.csv'), 'w')

arr_gdelt = df_gdelt.values



ind = 0
    # infoID : narrative name
    # k = window
    # n = number of days
for infoID in infoIDs:
	arr_event = df_event.loc[infoID,:].values
	w = df_corr.loc[infoID,:].values #
	W = []
	for j in range(n):
		W.append(w)

	for i in range(1,25):
		I = arr_gdelt[:,i+k:i+k+n].T # intended dates vs intended narra
		x = I*np.array(W)
		x = x.flatten()
		y = arr_event[i]

		fx_train.write(','.join(str(e) for e in x)+'\n')
		fi_train.write(str(ind)+'\n')
		fy_train.write(str(y)+'\n')


	id = infoID.replace('/','#')
	fx = open('./test/'+platform+'/'+id+'_x.csv', 'w')
	fi = open('./test/'+platform+'/'+id+'_ind.csv', 'w')
	fy = open('./test/'+platform+'/'+id+'_y.csv', 'w')
	fp = open('./test/'+platform+'/'+id+'_y_prev.csv', 'w')

	for i in range(25,39):
		I = arr_gdelt[:,i+k:i+k+n].T
		x = I*np.array(W)
		x = x.flatten()
		y = arr_event[i]
		fx.write(','.join(str(e) for e in x)+'\n')
		fi.write(str(ind)+'\n')
		fy.write(str(y)+'\n')
		print (infoID, str(y))

	ind += 1

	y_prev = arr_event[0:25]
	fp.write(','.join(str(e) for e in y_prev)+'\n')
	fx.close()
	fy.close()
	fp.close()

fx_train.close()
fy_train.close()

(47, 19)
(47, 39)
arrests 470
arrests 513
arrests 462
arrests 11018
arrests 26367
arrests 73161
arrests 47508
arrests 38521
arrests 50196
arrests 103278
arrests 50511
arrests 70033
arrests 59506
arrests 42813
arrests/opposition 162
arrests/opposition 263
arrests/opposition 147
arrests/opposition 5899
arrests/opposition 18799
arrests/opposition 65470
arrests/opposition 38112
arrests/opposition 32264
arrests/opposition 45931
arrests/opposition 95248
arrests/opposition 38174
arrests/opposition 64700
arrests/opposition 53832
arrests/opposition 37569
arrests/opposition/media 0
arrests/opposition/media 2
arrests/opposition/media 0
arrests/opposition/media 1
arrests/opposition/media 1814
arrests/opposition/media 3586
arrests/opposition/media 54
arrests/opposition/media 9
arrests/opposition/media 6
arrests/opposition/media 25
arrests/opposition/media 5
arrests/opposition/media 0
arrests/opposition/media 5948
arrests/opposition/media 588
arrests/opposition/protesters 2
arrests/opposition/protes

maduro/narco 1399
maduro/narco 1563
maduro/narco 1299
maduro/narco 3812
maduro/narco 2147
maduro/narco 22436
maduro/narco 9003
maduro/narco 4068
maduro/narco 4651
maduro/narco 9233
maduro/narco 1925
maduro/narco 2806
maduro/narco 4715
maduro/narco 7775
maduro/russia_support 76
maduro/russia_support 34
maduro/russia_support 20
maduro/russia_support 22
maduro/russia_support 35
maduro/russia_support 1244
maduro/russia_support 939
maduro/russia_support 1195
maduro/russia_support 1812
maduro/russia_support 4471
maduro/russia_support 733
maduro/russia_support 3173
maduro/russia_support 858
maduro/russia_support 118
military 6225
military 6077
military 7659
military 171564
military 108008
military 295387
military 220318
military 100658
military 71469
military 112290
military 82153
military 64502
military 92211
military 47085
military/desertions 67
military/desertions 67
military/desertions 32
military/desertions 25456
military/desertions 5078
military/desertions 7533
military/desertions 3169


In [8]:
arr_gdelt

array([[102,  66,  83, ..., 302, 384, 342],
       [ 11,   4,  24, ...,  59,  78,  68],
       [101, 135, 102, ..., 326, 319, 253],
       ...,
       [  9,   3,   9, ...,  72,  52,  36],
       [ 34,  27,  20, ..., 140, 142,  74],
       [ 50,  22,  37, ...,  97, 110, 115]])

In [9]:
arr = [  77655.91153717,   69317.83901978,   68188.94837952,  504752.35339355,
  568800.90869141 ,2878337.87036133, 1578978.26062012,  665663.09155273,
  718088.80151367 , 796427.9967041  , 535527.5874939  , 466776.96887207,
  489436.62670898,  394332.40637207]
sum(arr)

9812285.5712204

In [10]:
df_corr

,010,012,020,030,036,040,042,043,046,051,100,110,111,112,120,130,141,173,190
arrests,0.742519,0.702280,0.783579,0.709375,0.685473,0.709530,0.777985,0.762311,0.768763,0.741883,0.773855,0.787700,0.664669,0.818703,0.781178,0.757874,0.703411,0.484389,0.785075
arrests/opposition,0.724108,0.685944,0.765124,0.689937,0.676099,0.695247,0.767071,0.750509,0.757743,0.723720,0.763545,0.767239,0.645607,0.802664,0.769073,0.747034,0.676808,0.473654,0.756911
arrests/opposition/media,0.000000,0.000000,0.000000,0.000000,0.391717,0.000000,0.338488,0.335652,0.327344,0.000000,0.000000,0.000000,0.000000,0.322018,0.000000,0.302505,0.417917,0.381427,0.380990
arrests/opposition/protesters,0.631610,0.549090,0.647537,0.607659,0.462455,0.536443,0.624101,0.608918,0.607691,0.684577,0.681605,0.673290,0.549518,0.703996,0.697564,0.627042,0.623416,0.000000,0.658207
assembly/illegitimate,0.322879,0.000000,0.348872,0.351114,0.348777,0.000000,0.000000,0.000000,0.305589,0.000000,0.422648,0.313252,0.362132,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
assembly/legitimate,0.306468,0.000000,0.000000,0.330231,0.000000,0.000000,0.000000,0.000000,0.000000,0.306264,0.300442,0.334881,0.000000,0.000000,0.000000,0.000000,0.560593,0.000000,0.435452
crisis,0.533021,0.427285,0.552792,0.562323,0.416814,0.438296,0.536434,0.521275,0.498715,0.593090,0.602807,0.566980,0.537366,0.560579,0.520034,0.491500,0.694615,0.366601,0.657983
crisis/lack_essentials,0.516752,0.431753,0.560387,0.559103,0.453679,0.453396,0.542916,0.525534,0.518064,0.580984,0.635188,0.552740,0.545289,0.552985,0.510288,0.484337,0.652986,0.386527,0.649192
crisis/looting,0.577760,0.489195,0.447041,0.606519,0.000000,0.000000,0.338207,0.369950,0.336270,0.601007,0.397745,0.525074,0.584930,0.423920,0.377932,0.460433,0.660791,0.000000,0.535193
guaido/illegitimate,0.905711,0.931660,0.930455,0.919896,0.902736,0.886416,0.924235,0.921308,0.945755,0.881715,0.877500,0.904333,0.868738,0.918256,0.863503,0.925476,0.820900,0.704342,0.883672


In [16]:
df_corr.loc[:,"other/censorship_outage"]

KeyError: 'other/censorship_outage'